***Week - 9 : Assignment - NLP POS Substitutions and Tones for Poets***

**Team :** Data Miners

**Author :** Poojashree NS

In [ ]:
#pip install beautifulsoup4

In [ ]:
#!pip install -U pandas-profiling

In [ ]:
import warnings
warnings.filterwarnings('ignore')

**Import Libraries**

In [ ]:
import urllib.request
import numpy as np
import pandas as pd
import bs4 as bs
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
# Format the scraped code
def beautify_txt(text):
    #final_data = (((text).replace(u'\xa0', u' ')).replace(r'\r',u'\n'))
    final_data = (((text).replace(u'\xa0', u' ')).replace(u'\r',u'\n'))

    return final_data

In [ ]:
# Poet and Poems
Edgar_Allan_Poe_Poems=["https://allpoetry.com/poem/8448397-A-Dream-by-Edgar-Allan-Poe", "https://allpoetry.com/The-Happiest-Day-The-Happiest-Hour"]
Christina_Georgina_Rossetti_Poem=["https://allpoetry.com/A-Peal-Of-Bells", "https://allpoetry.com/poem/8449115-A-Dream-by-Christina-Georgina-Rossetti"]

**Scraping data, format poem, preform preprocessing and storing in the form of dataframe**

In [ ]:
def scrape_poem(url):
  poet_list = []
  poem_pos = []

  header = {'User-Agent':'Mozilla/5.0'}
  requests = urllib.request.Request(url,headers=header)
  requestread = urllib.request.urlopen(requests).read()
  soup = bs.BeautifulSoup(requestread,'html.parser')

  poem_content = (beautify_txt(soup.find_all('div', class_="poem_body")[0].text))
  poem_title = soup.find_all('h1')[0].text
  
  word_list = word_tokenize(poem_content)
  rev = []
  lemmatizer = WordNetLemmatizer() 
  #stemmer = PorterStemmer() 
  for word, tag in pos_tag(word_list):
    if tag.startswith('J'):
      w = lemmatizer.lemmatize(word, pos='a')
    elif tag.startswith('V'):
      w = lemmatizer.lemmatize(word, pos='v')
    elif tag.startswith('N'):
      w = lemmatizer.lemmatize(word, pos='n')
    elif tag.startswith('R'):
      w = lemmatizer.lemmatize(word, pos='r')
    else:
      w = word
    #w = stemmer.stem(w)
    rev.append(w)
  poem_content_cleaned = ' '.join(rev)

  tokenized = sent_tokenize(poem_content)
  for i in tokenized[:-2]:
    # Word tokenizers is used to find the words
    # and punctuation in a string
    wordsList = nltk.word_tokenize(i)

    #Way to eliminate special characters and puncuations. 
    wordsList=[word.lower() for word in wordsList if word.isalpha()]
      
    # removing stop words from wordList
    wordsList = [w for w in wordsList if not w in stop_words]
 
    #  Using a Tagger. Which is part-of-speech
    # tagger or POS-tagger.
    tagged = nltk.pos_tag(wordsList)
  
    for x in tagged:
      poem_pos.append(x)

  poet_list.append([poem_title, poem_content, poem_pos])
  df = pd.DataFrame(poet_list)
  df.columns = ['poem_title', 'poem','poem_pos']
  return df

**Combining all poems into 1 dataset**

In [ ]:
df_EAP_1 = scrape_poem(Edgar_Allan_Poe_Poems[0])
df_EAP_2 = scrape_poem(Edgar_Allan_Poe_Poems[1])
df_CGR_1 = scrape_poem(Christina_Georgina_Rossetti_Poem[0])
df_CGR_2 = scrape_poem(Christina_Georgina_Rossetti_Poem[1])

In [ ]:
dfs = [df_EAP_1, df_EAP_2, df_CGR_1, df_CGR_2]
final_df = pd.concat(dfs, ignore_index=True)

In [ ]:
poet = ['Edgar_Allan_Poe_1', 'Edgar_Allan_Poe_2', 'Christina_Georgina_Rossetti_1', 'Christina_Georgina_Rossetti_2']
final_df['poet'] = poet

In [ ]:
def print_poem_aligned(poem_df):
  for col_name, data in poem_df.poem.items():
    poem_sample = data.split('\n')
    for line in poem_sample:
      print(f"{line}")

In [ ]:
print_poem_aligned(final_df)

In visions of the dark night

        I have dreamed of joy departed-

      But a waking dream of life and light

        Hath left me broken-hearted.


      Ah! what is not a dream by day

        To him whose eyes are cast

      On things around him with a ray

        Turned back upon the past?


      That holy dream- that holy dream,

        While all the world were chiding,

      Hath cheered me as a lovely beam

        A lonely spirit guiding.


      What though that light, thro' storm and night,

        So trembled from afar-

      What could there be more purely bright

        In Truth's day-star? © by owner. provided at no charge for educational purposes    
The happiest day- the happiest hour
        My sear'd and blighted heart hath known,
      The highest hope of pride and power,
        I feel hath flown.

      Of power! said I? yes! such I ween;
        But they have vanish'd long, alas!
      The visions of my youth have been-
        But let them pass.

   

**Final Dataset**

In [ ]:
final_df

,poem_title,poem,poem_pos,poet
0,A Dream,In visions of the dark night\n\n I have...,"[(visions, NNS), (dark, VBP), (night, NN), (dr...",Edgar_Allan_Poe_1
1,The Happiest Day-The Happiest Hour,The happiest day- the happiest hour\n M...,"[(happiest, JJS), (happiest, JJS), (hour, NN),...",Edgar_Allan_Poe_2
2,A Peal Of Bells,"Strike the bells wantonly,\n Tinkle tinkle we...","[(strike, NN), (bells, NNS), (wantonly, RB), (...",Christina_Georgina_Rossetti_1
3,A Dream,Once in a dream (for once I dreamed of you)\n ...,"[(dream, NN), (dreamed, VBD), (stood, VBD), (t...",Christina_Georgina_Rossetti_2


**Get Noun, Adjective and Verb for each poem and store it in dataframe**

In [ ]:
def df_hashmap(data, poet, dataframe):
  data_hf = {}
  value_list = []

  df_pos = pd.DataFrame.from_dict(data)
  for ind in df_pos.index:
    data_hf.setdefault(df_pos[1][ind], []).append(df_pos[0][ind])
  
  noun_list = []
  adj_list = []
  vb_list = []

  for key in data_hf:
    if key.startswith('N'):
      noun_list = noun_list + data_hf[key]
     
    if key.startswith('J'):
      adj_list = adj_list + data_hf[key]

    if key.startswith('V'):
      vb_list = vb_list + data_hf[key]

  final_pos = noun_list + adj_list + vb_list
  pos_dict = {}
  pos_dict.setdefault("POS", []).append(final_pos)

  df = pd.DataFrame(pos_dict.items())
  df = df.T
  df.columns = df.iloc[0] 
  df = df[1:]
  df['poet'] = poet
  df_final = pd.merge(dataframe, df)
  
  return df_final

In [ ]:
EAP_POS_data_1 = df_hashmap(final_df.poem_pos[0], final_df.poet[0], final_df)
EAP_POS_data_2 = df_hashmap(final_df.poem_pos[1], final_df.poet[1], final_df)
CGR_POS_data_1 = df_hashmap(final_df.poem_pos[2], final_df.poet[2], final_df)
CGR_POS_data_2 = df_hashmap(final_df.poem_pos[3], final_df.poet[3], final_df)

In [ ]:
dfs = [EAP_POS_data_1, EAP_POS_data_2, CGR_POS_data_1, CGR_POS_data_2]
df_final = pd.concat(dfs, ignore_index=True)

**Converting dataframe into JSON and csv file**

In [ ]:
df_final.to_json('poet.json', orient ='records', compression = 'infer', index = 'true')

In [ ]:
df_final.to_csv('poet_data.csv')

In [ ]:
df_final

,poem_title,poem,poem_pos,poet,POS
0,A Dream,In visions of the dark night\n\n I have...,"[(visions, NNS), (dark, VBP), (night, NN), (dr...",Edgar_Allan_Poe_1,"[[visions, eyes, things, night, joy, life, hat..."
1,The Happiest Day-The Happiest Hour,The happiest day- the happiest hour\n M...,"[(happiest, JJS), (happiest, JJS), (hour, NN),...",Edgar_Allan_Poe_2,"[[hour, sear, heart, hath, hope, pride, power,..."
2,A Peal Of Bells,"Strike the bells wantonly,\n Tinkle tinkle we...","[(strike, NN), (bells, NNS), (wantonly, RB), (...",Christina_Georgina_Rossetti_1,"[[strike, wine, bring, silver, bell, oil, hung..."
3,A Dream,Once in a dream (for once I dreamed of you)\n ...,"[(dream, NN), (dreamed, VBD), (stood, VBD), (t...",Christina_Georgina_Rossetti_2,"[[dream, field, ease, view, darkness, hawk, fi..."


**Replace Poet_A POS with Poet_B POS and Vice-Versa considering Semantic similarities. Finally store them into text file**

In [ ]:
# !pip install spacy
#!python3 -m spacy download en_core_web_sm

In [ ]:
from numpy.ma.core import greater
import itertools
import spacy
  
nlp = spacy.load('en_core_web_sm')

def get_converted_poem(file_name , num_1, num_2):
  wordlist_1 = df_final['POS'][num_1]
  wordlist_2 = df_final['POS'][num_2]
  list_1 = set(itertools.chain(*wordlist_1))
  list_2 = set(itertools.chain(*wordlist_2))

  hashmap = {}
  for word_1 in list_1:
    pre_similarity = 0.3
    for word_2 in list_2:
      words = word_1 + " " + word_2
      
      tokens = nlp(words)

      token1, token2 = tokens[0], tokens[1]
      similarity = token1.similarity(token2)
      if similarity > pre_similarity:
        hashmap[word_1] = word_2
        pre_similarity = similarity

  poem_in = df_final.poem[num_1]
  poem_out = re.sub(r'\b(\w+)\b', lambda m:hashmap.get(m.group(1), m.group(1)), poem_in.lower())

  file1 = open(file_name,"w")
  file1.writelines(poem_out)

In [ ]:
get_converted_poem("EAP1_CGR1.txt", 0 , 2)
get_converted_poem("EAP1_CGR2.txt", 0 , 3)

get_converted_poem("EAP2_CGR2.txt", 1 , 2)
get_converted_poem("EAP2_CGR1.txt", 1 , 3)

get_converted_poem("CGR1_EAP1.txt", 2 , 0)
get_converted_poem("CGR1_EAP2.txt", 2 , 1)

get_converted_poem("CGR2_EAP1.txt", 3 , 0)
get_converted_poem("CGR2_EAP2.txt", 3 , 1)